In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('Mnist_data/', one_hot=True)

Extracting Mnist_data/train-images-idx3-ubyte.gz
Extracting Mnist_data/train-labels-idx1-ubyte.gz
Extracting Mnist_data/t10k-images-idx3-ubyte.gz
Extracting Mnist_data/t10k-labels-idx1-ubyte.gz


## CNN Steps
1. Helper
2. INIT Weight
3. INIT Bias
4. CONV 2D
5. Pooling

In [4]:
# Helper

def init_weight(shape):
    init_random_weight = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_weight)

def init_bais(shape):
    init_random_bais = tf.constant(0.1, shape=shape)
    return tf.Variable(init_random_bais)

In [5]:
# CONV 2D

def conv2d(x, W):
    # x --> [batch, H, W, Channels]
    # W --> [FIlter H, filter W, Channel IN, Channel Out]
    
    return tf.nn.conv2d(input=x, filter=W, strides=[1,1,1,1], padding='SAME')

In [6]:
# Polling

def max_pool(x):
    # x --> same as conv 2d
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
# CONV Layer

def conv_layer(input_x, shape):
    W = init_weight(shape)
    b = init_bais([shape[3]])
    
    return tf.nn.relu(conv2d(input_x, W) + b)

In [8]:
# Fully Conected
def normal_fully_connected(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weight([input_size, size])
    b = init_bais([size])
    return tf.matmul(input_layer, W) + b

## Building CNN 

In [9]:
# Placeholder
x = tf.placeholder(tf.float32,shape=[None,784])
y_true = tf.placeholder(tf.float32, shape=[None,10])

In [10]:
# Layers
x_image = tf.reshape(x, shape=[-1, 28, 28, 1])

In [11]:
convo_1 = conv_layer(x_image,shape=[6,6,1,32])
convo_1_pooling = max_pool(convo_1)

convo_2 = conv_layer(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool(convo_2)

convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_fully_connected(convo_2_flat,1024))

In [12]:
# NOTE THE PLACEHOLDER HERE!
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [13]:
y_pred = normal_fully_connected(full_one_dropout, 10)

In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [15]:
# Optimizer
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(cross_entropy)

In [16]:
init = tf.global_variables_initializer()

In [17]:
steps = 500

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(100)
        
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%10 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            batch_x_test , batch_y_test = mnist.train.next_batch(100)
            
            print(sess.run(acc,feed_dict={x:batch_x_test,y_true:batch_y_test,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.17


Currently on step 10
Accuracy is:
0.6


Currently on step 20
Accuracy is:
0.79


Currently on step 30
Accuracy is:
0.91


Currently on step 40
Accuracy is:
0.88


Currently on step 50
Accuracy is:
0.97


Currently on step 60
Accuracy is:
0.96


Currently on step 70
Accuracy is:
0.95


Currently on step 80
Accuracy is:
0.95


Currently on step 90
Accuracy is:
0.91


Currently on step 100
Accuracy is:
0.96


Currently on step 110
Accuracy is:
1.0


Currently on step 120
Accuracy is:
0.96


Currently on step 130
Accuracy is:
0.99


Currently on step 140
Accuracy is:
0.98


Currently on step 150
Accuracy is:
0.92


Currently on step 160
Accuracy is:
0.96


Currently on step 170
Accuracy is:
0.96


Currently on step 180
Accuracy is:
0.97


Currently on step 190
Accuracy is:
0.97


Currently on step 200
Accuracy is:
0.96


Currently on step 210
Accuracy is:
0.96


Currently on step 220
Accuracy is:
0.97


Currently on step 230
Accuracy is:
0.97


Curre